In [ ]:
import yaml
import sys
sys.path.append("../../src")
from images import create_train_test
from images import to_tensorflow
from images import augment_image
from images import learning_curves
from images import create_model
from keras.utils import plot_model
import os
import tensorflow as tf

## Geotags soft f1

In [ ]:
with open("../../ml_models/images/image_model.yaml", "r") as f:
    image_model_config = yaml.safe_load(f)

INPUT_SHAPE = (224,224,3)

data_path = "../../data/processed/data.json"
#This splits the recepies into train and test. This doesn't actually load the features of the data
X_train, X_val, y_train, y_val = create_train_test(image_model_config,data_path, "geographical_tags_updated")

#This part loads in the data as tensorflow objects
train_ds, val_ds, N_LABELS, y_val_bin, mlb = to_tensorflow(X_train, X_val, y_train, y_val, image_model_config)

In [ ]:
# Apply data augmentation to the training dataset
train_ds_augmented = train_ds.map(augment_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Unbatch the augmented dataset
train_ds = train_ds_augmented.unbatch()

# Apply data augmentation to the training dataset
val_ds_augmented = val_ds.map(augment_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Unbatch the augmented dataset
val_ds = val_ds_augmented.unbatch()

model_path = f"../../ml_models/images/"
number = str(len(os.listdir(model_path)))
model_path += f"final_model_{image_model_config['EPOCHS']}_epochs"+ ".h5"

In [ ]:
model = create_model(image_model_config, INPUT_SHAPE, N_LABELS)

In [ ]:
#plot_model(model, show_shapes=True)

In [ ]:
history = model.fit(train_ds,
                    epochs=int(image_model_config["EPOCHS"]),
                    validation_data=val_ds)

In [ ]:
model.save_weights(model_path)

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.style as style
rc = {'axes.facecolor':'white',
      'axes.grid' : False,
      'axes.spines.bottom': False,
      'axes.spines.left': False,
      'axes.spines.right': False,
      'axes.spines.top': False,
      'savefig.transparent': False}

def learning_curves(history):
    """Plot the learning curves of loss and macro f1 score 
    for the training and validation datasets.
    
    Args:
        history: history callback of fitting a tensorflow keras model 
    """
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    macro_f1 = history.history['macro_f1']
    val_macro_f1 = history.history['val_macro_f1']
    
    epochs = len(loss)

    plt.rcParams.update(rc)

    plt.figure(figsize=(18, 5))  # Increase the figure height

    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs+1), loss, label='Training Loss')
    plt.plot(range(1, epochs+1), val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Loss')
    plt.xlabel("Epochs")
    plt.title('Training and Validation Loss')
    
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs+1), macro_f1, label='Training Macro F1-score')
    plt.plot(range(1, epochs+1), val_macro_f1, label='Validation Macro F1-score')
    plt.legend(loc='lower right')
    plt.ylabel('Macro F1-score')
    plt.xlabel("Epochs")
    plt.title('Training and Validation Macro F1-score')

    #plt.subplots_adjust(hspace=0.2)  # Adjust the vertical space between subplots
    plt.savefig("../../visualizations/images/training_analysis.png", dpi = 300, bbox_inches = "tight")
    plt.show()

    
    return loss, val_loss, macro_f1, val_macro_f1


In [ ]:
model_bce_losses, model_bce_val_losses, model_bce_macro_f1s, model_bce_val_macro_f1s = learning_curves(history)